In [351]:
import os
import json
import numpy as np
import tensorflow as tf

In [352]:
# List that holds all the grids to be used as inputs for training
x_train_grid = []
# List that holds all the positions to be used as inputs for training
x_train_position = []
# List that holds all the distances to be used as inputs for training
x_train_distance = []
# List that holds all the directions to be used as outputs for training
y_train = []

# List that holds all the grids to be used as inputs for testing
x_test_grid = []
# List that holds all the positions to be used as inputs for testing
x_test_position = []
# List that holds all the distances to be used as inputs for testing
x_test_distance = []
# List that holds all the directions to be used as outputs for testing
y_test = []

In [353]:
# Name of directory for grids
directory_name = './data/small_grids/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_train_grid.append(i['gridworld'])
        x_train_position.append(i['position'])
        y_train.append(i['direction'])
    
    # Close file socket
    f.close()

In [354]:
# Name of directory for grids
directory_name = './data/small_grids_test/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_test_grid.append(i['gridworld'])
        x_test_position.append(i['position'])
        y_test.append(i['direction'])
    
    # Close file socket
    f.close()

In [355]:
print(x_train_position)
print(y_train)

[[0, 0], [1, 0], [2, 0], [3, 0], [3, 0], [3, 1], [4, 1], [5, 1], [6, 1], [7, 1], [8, 1], [9, 1], [9, 1], [8, 1], [8, 2], [8, 3], [8, 3], [7, 3], [7, 4], [7, 4], [6, 4], [6, 4], [5, 4], [5, 5], [5, 6], [6, 6], [7, 6], [8, 6], [9, 6], [10, 6], [11, 6], [12, 6], [12, 6], [11, 6], [11, 7], [11, 7], [10, 7], [10, 8], [10, 9], [10, 9], [9, 9], [9, 10], [9, 10], [8, 10], [8, 11], [8, 11], [7, 11], [7, 11], [6, 11], [6, 12], [6, 13], [7, 13], [8, 13], [9, 13], [9, 13], [9, 14], [10, 14], [11, 14], [12, 14], [13, 14], [14, 14], [15, 14], [16, 14], [17, 14], [18, 14], [19, 14], [20, 14], [21, 14], [22, 14], [23, 14], [24, 14], [25, 14], [25, 14], [24, 14], [24, 15], [24, 15], [24, 14], [23, 14], [22, 14], [22, 15], [22, 16], [22, 16], [22, 17], [23, 17], [23, 17], [23, 18], [24, 18], [25, 18], [26, 18], [27, 18], [28, 18], [28, 18], [28, 19], [29, 19], [29, 20], [29, 21], [29, 21], [28, 21], [28, 22], [28, 23], [29, 23], [29, 24], [29, 25], [29, 26], [29, 27], [29, 28], [0, 0], [1, 0], [2, 0], [

In [356]:
# Reshape the data
train_in_grid = np.reshape( x_train_grid, (-1, 30, 30) ) / 2
train_in_position = np.reshape( x_train_position, (-1, 2, 1) ) / 2
train_out = tf.keras.utils.to_categorical( y_train, 4 )

test_in_grid = np.reshape( x_test_grid, (-1, 30, 30) ) / 2
test_in_position = np.reshape( x_test_position, (-1, 2, 1) ) / 2
test_out = tf.keras.utils.to_categorical( y_test, 4 )

In [357]:
# Flatten the grid input
grid_input = tf.keras.layers.Input( shape = (30,30) )
flatten_grid = tf.keras.layers.Flatten()( grid_input )

In [358]:
# Flatten the position input
position_input = tf.keras.layers.Input( shape = (2,1) )
flatten_position = tf.keras.layers.Flatten()( position_input )

In [359]:
# Concatenate the grid and position into one vector which will be passed to neural network as input
final_input = tf.keras.layers.Concatenate()([flatten_grid, flatten_position])

In [360]:
# Create layers for Neural Network
dense_1 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( final_input )
dense_2 = tf.keras.layers.Dense( units = 50, activation = tf.nn.relu )( dense_1 )
dense_3 = tf.keras.layers.Dense( units = 10, activation = tf.nn.relu )( dense_2 )
logits = tf.keras.layers.Dense( units = 4, activation = None )( dense_3 )
probabilities = tf.keras.layers.Softmax()( logits )

In [361]:
# Compile the neural network to use stochastic gradient descent as the optimizer and categorical_crossentropy as loss function
model = tf.keras.Model( inputs = [grid_input, position_input], outputs = probabilities )
model.compile( optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [362]:
def generate_confusion_matrix( data, labels ):
    mat = [ [ 0 for i in range(4) ] for j in range(4) ]
    
    predictions = np.argmax( model.predict( data ), axis = 1 )
    
    for i in range( data[0].shape[0] ):
        mat[ labels[i] ][ predictions[i] ] += 1
    
    for i in range(4):
        print( "\t".join( [ str(c) for c in mat[i] ] ) )

In [363]:
# Test out before training
generate_confusion_matrix( [test_in_grid, test_in_position], y_test )

0	0	0	8
0	0	1	15
0	0	3	40
0	0	2	61


In [364]:
# Train the model
history = model.fit( [train_in_grid, train_in_position], train_out, epochs = 20 )

Epoch 1/20
39/39 [==============================] - 0s 2ms/step - loss: 1.1693 - accuracy: 0.4727
Epoch 2/20
39/39 [==============================] - 0s 2ms/step - loss: 1.1354 - accuracy: 0.4839
Epoch 3/20
39/39 [==============================] - 0s 2ms/step - loss: 1.1269 - accuracy: 0.4960
Epoch 4/20
39/39 [==============================] - 0s 2ms/step - loss: 1.1184 - accuracy: 0.4968
Epoch 5/20
39/39 [==============================] - 0s 2ms/step - loss: 1.1147 - accuracy: 0.5048
Epoch 6/20
39/39 [==============================] - 0s 4ms/step - loss: 1.1083 - accuracy: 0.5096
Epoch 7/20
39/39 [==============================] - 0s 5ms/step - loss: 1.1053 - accuracy: 0.5193
Epoch 8/20
39/39 [==============================] - 0s 4ms/step - loss: 1.0921 - accuracy: 0.5088
Epoch 9/20
39/39 [==============================] - 0s 5ms/step - loss: 1.1029 - accuracy: 0.5249
Epoch 10/20
39/39 [==============================] - 0s 7ms/step - loss: 1.1012 - accuracy: 0.5177
Epoch 11/20
39/39 [

In [365]:
# Test out after training
generate_confusion_matrix( [test_in_grid, test_in_position], y_test )

0	0	6	2
0	0	15	1
0	0	31	12
0	0	21	42
